**Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
# %cd /gdrive/MyDrive/AN2DL/Homework2

### Configurations

**Get Data Sets**

In [ ]:
!wget -q https://storage.googleapis.com/storage.barbiero.dev/AN2DL/Homework_2/training_data_clean.npy
!wget -q https://storage.googleapis.com/storage.barbiero.dev/AN2DL/Homework_2/categories_clean.npy
!wget -q https://storage.googleapis.com/storage.barbiero.dev/AN2DL/Homework_2/series_length_clean.npy

**Update Tensorflow**

In [ ]:
!pip install --upgrade tensorflow -q

**Imports**

In [ ]:
# Deafault Imports
import os
import logging
import warnings as wr
import numpy as np
import random as rnd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras as tfk
from keras import layers as tkl
from keras import models as tkm
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# from statsmodels.tsa.seasonal import seasonal_decompose
# from dateutil.parser import parse
# from statsmodels.tsa.stattools import adfuller

**Randomness & Warinings**

In [ ]:
# Random Configuration - All
RND = False
if not RND:
  SEED = 42
  os.environ['PYTHONHASHSEED'] = str(SEED)
  tf.compat.v1.set_random_seed(SEED)
  tf.random.set_seed(SEED)
  np.random.seed(SEED)
  rnd.seed(SEED)

# OS Configuration
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

# Warning Congiguration
wr.simplefilter(action='ignore', category=FutureWarning)
wr.simplefilter(action='ignore', category=Warning)

# TensorFlow Configuration
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Plotting Configuration
plt.rc('font', size=16)

**TPU Config**

In [ ]:
use_tpu = True

if 'COLAB_TPU_ADDR' in os.environ and use_tpu:
  TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])
else:
  use_tpu = False

if use_tpu:
  tpu_address = TF_MASTER
  resolver = tf.distribute.cluster_resolver.TPUClusterResolver(TF_MASTER)
  tf.config.experimental_connect_to_cluster(resolver)
  tf.tpu.experimental.initialize_tpu_system(resolver)
  strategy = tf.distribute.TPUStrategy(resolver)

**Initiate Data Sets**

In [ ]:
# # Loading Datasets RAW
# TD = np.load("/gdrive/MyDrive/AN2DL/Homework2/training_data.npy", allow_pickle=True)
# VP = np.load("/gdrive/MyDrive/AN2DL/Homework2/valid_periods.npy", allow_pickle=True)
# CG = np.load("/gdrive/MyDrive/AN2DL/Homework2/categories.npy", allow_pickle=True)

# Loading Datasets CLEAN
TD = np.load("training_data_clean.npy", allow_pickle=True)
VP = np.load("series_length_clean.npy", allow_pickle=True)
CG = np.load("categories_clean.npy", allow_pickle=True)

**Data Frame**

In [ ]:
# Create Dataframe Unified CLEAN
DATA = []

for i, l in enumerate(VP):
  ts_clipped = TD[i, :l]
  DATA.append(ts_clipped)

df = pd.DataFrame({
    'TimeSeries': [ts.tolist() for ts in DATA],
    'Category': CG.flatten(),
    'Length': VP.flatten()
})
df['Category'] = df['Category'].map({0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F'})

dft = df['TimeSeries']
dfc = df['Category']

print(df['Category'].value_counts())

**Stats of the Data**

In [ ]:
df_cat = pd.DataFrame()
dd = pd.DataFrame()
# Avarage Length
avg_length_cat = df.groupby('Category')['Length'].mean().round(2)
df_cat['AVG Length'] = avg_length_cat

# Less than 50 Elements
threshold = 50
df_cat['Below 50 (pcs)'] = df.groupby('Category')['Length'].apply(lambda x: (x < threshold).sum())
df_cat['Below 50 (%)'] = df.groupby('Category')['Length'].apply(lambda x: (x<threshold).mean()*100).round(2)

# Less than 75 Elements
threshold = 75
df_cat['Below 75 (pcs)'] = df.groupby('Category')['Length'].apply(lambda x: (x<threshold).sum())
df_cat['Below 75 (%)'] = df.groupby('Category')['Length'].apply(lambda x: (x<threshold).mean()*100).round(2)

# Less than 100 Elements
threshold = 100
df_cat['Below 100 (pcs)'] = df.groupby('Category')['Length'].apply(lambda x: (x<threshold).sum())
df_cat['Below 100 (%)'] = df.groupby('Category')['Length'].apply(lambda x: (x<threshold).mean()*100).round(2)

# Less than 150 Elements
threshold = 150
df_cat['Below 150 (pcs)'] = df.groupby('Category')['Length'].apply(lambda x: (x<threshold).sum())
df_cat['Below 150 (%)'] = df.groupby('Category')['Length'].apply(lambda x: (x<threshold).mean()*100).round(2)

# Less than 200 Elements
threshold = 200
df_cat['Below 200 (pcs)'] = df.groupby('Category')['Length'].apply(lambda x: (x<threshold).sum())
df_cat['Below 200 (%)'] = df.groupby('Category')['Length'].apply(lambda x: (x<threshold).mean()*100).round(2)

# Less than 218 Elements
threshold = 218
df_cat['Below 218 (pcs)'] = df.groupby('Category')['Length'].apply(lambda x: (x<threshold).sum())
df_cat['Below 218 (%)'] = df.groupby('Category')['Length'].apply(lambda x: (x<threshold).mean()*100).round(2)

# Less than 536 Elements
threshold = 536
df_cat['Below 536 (pcs)'] = df.groupby('Category')['Length'].apply(lambda x: (x<threshold).sum())
df_cat['Below 536 (%)'] = df.groupby('Category')['Length'].apply(lambda x: (x<threshold).mean()*100).round(2)

print(df_cat[['AVG Length']])
print()
print(df_cat[['Below 50 (pcs)', 'Below 50 (%)']])
print(df_cat[['Below 75 (pcs)', 'Below 75 (%)']])
print(df_cat[['Below 100 (pcs)', 'Below 100 (%)']])
print(df_cat[['Below 150 (pcs)', 'Below 150 (%)']])
print(df_cat[['Below 200 (pcs)', 'Below 200 (%)']])
print(df_cat[['Below 218 (pcs)', 'Below 218 (%)']])
print(df_cat[['Below 536 (pcs)', 'Below 536 (%)']])

**Category Dataframes**

In [ ]:
# Dataframes
df_A = df[df['Category'] == 'A']
df_B = df[df['Category'] == 'B']
df_C = df[df['Category'] == 'C']
df_D = df[df['Category'] == 'D']
df_E = df[df['Category'] == 'E']
df_F = df[df['Category'] == 'F']

df_AB = pd.concat([df_A, df_B], ignore_index=True)

### Functions ###

In [ ]:
# Time Series Plot Functions
def print_timeseries(n, random, norm):
  for i in range(n):
    if random:
      j = np.random.randint(0,len(df))
    else:
      j=i
    print(dfc[j], "Length: ",len(dft[j]))
    if norm:
      plt.plot(range(len(dft[j])), [x*100 for x in dft[j]], label=f'Time Series {i + 1}')
    else:
      plt.plot(range(len(dft[j])), dft[j], label=f'Time Series {i + 1}')

    plt.title(f'#{j}')
    plt.xlabel('Time Index')
    plt.ylabel('Value')
    # plt.legend()
    plt.show()

    return j
def print_category(n, rows, cols, category):
  indices = []

  if n > rows*cols:
    print("Please add more rooms")
    return

  fig, axes = plt.subplots(rows, cols, figsize=(15*cols,5*rows))
  for i in range(n):
    j = np.random.randint(0, len(df))

    while dfc[j] != category or j in indices:
      j = np.random.randint(0, len(df))

    row_i = i // cols
    col_i = i % cols

    print(dfc[j], "Length: ",len(dft[j]))
    axes[row_i, col_i].plot(range(len(dft[j])), dft[j], label=f'Time Series {i + 1}')
    axes[row_i, col_i].set_title(f'#{j}')
    axes[row_i, col_i].set_xlabel('Time Index')
    axes[row_i, col_i].set_ylabel('Value')

    indices.append(j)

  plt.tight_layout()
  plt.show()
  

In [ ]:
# "Augmentation" CLEAN
def build_sequence(df, window=200, stride=200, telescope=18):
  actual_window = window + telescope
  new_categories = []
  X = []                                                                        # 2d (number of series, window size)
  y = []                                                                        # 2d (number of series, telescope)

  for i in range(len(df)):
    ts = df['TimeSeries'][i]
    length = df['Length'][i]
    category = df['Category'][i]

    new_stride = stride
    n_windows = int(np.ceil((length - actual_window) / new_stride)) + 1         # number of windows
    if n_windows < 1:
      n_windows = 1
    if n_windows > 1:                                                           # evalute the stride again
      new_stride = int((length - actual_window) / (n_windows - 1))

    start_idx = length - actual_window                                          # start from the end of the series
    end_idx = length
    for j in range(n_windows):
      if start_idx < 0:
        start_idx = 0
        end_idx = actual_window
        if end_idx > length:
          end_idx = length

      X.append(ts[start_idx:end_idx - telescope])
      y.append(ts[end_idx - telescope:end_idx])
      new_categories.append(category)

      start_idx -= new_stride
      end_idx -= new_stride

  return np.array(X), np.array(y), new_categories


In [ ]:
def build_CONV_LSTM_model(input_shape, output_shape):
    assert input_shape[0] >= output_shape[0], "For this exercise we want input time steps to be >= of output time steps"

    # Define the input layer with the specified shape
    input_layer = tkl.Input(shape=input_shape, name='input_layer')

    # Add a Bidirectional LSTM layer with 64 units
    x = tkl.Bidirectional(tkl.LSTM(input_shape[0], return_sequences=True, name='lstm_1'), name='bidirectional_lstm_1')(input_layer)
    x = tkl.MultiHeadAttention(num_heads=1, key_dim=input_shape[0], dropout=0.2)(x, x)
    x = tkl.Bidirectional(tkl.LSTM(input_shape[0], return_sequences=True, name='lstm_2'), name='bidirectional_lstm_2')(x)
    x = tkl.Dense(units=output_shape[1], activation = 'linear')(x)
    x = tkl.Flatten()(x)
    x = tkl.Dense(units=output_shape[0])(x)

    output_layer = tkl.Reshape((-1, 1))(x)

    # Construct the model by connecting input and output layers
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer, name='CONV_LSTM_model')

    # Compile the model with Mean Squared Error loss and Adam optimizer
    model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam())

    return model

### Training ###

In [ ]:
XTR, XTE = train_test_split(df_A, test_size=0.20, stratify=df_A['Category'], random_state=SEED)
XTR = XTR.reset_index(drop=True)
XTE = XTE.reset_index(drop=True)

print(XTR)

In [ ]:
WIN = 50
TEL = 18
STR = 25
ATL = 3

XTR_ = XTR[XTR['Length'] >= WIN+TEL]
XTE_ = XTR[XTR['Length'] >= WIN+TEL]
XTR_ = XTR_.reset_index(drop=True)
XTE_ = XTE_.reset_index(drop=True)

X_train, y_train, cat_train = build_sequence(df=XTR_, window=WIN, stride=STR, telescope=TEL)
X_test, y_test, cat_test = build_sequence(df=XTE_, window=WIN, stride=STR, telescope=TEL)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [ ]:
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)
y_train = np.expand_dims(y_train, axis=-1)
y_test = np.expand_dims(y_test, axis=-1)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [ ]:
# Assign the batch size
if use_tpu:
  BATCH_SIZE = 128
else:
  BATCH_SIZE = 128
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
def prepare_trainset(dataset):
    return (
        dataset
        .cache()
        .shuffle(1000, reshuffle_each_iteration=True)
        .repeat()
        .batch(BATCH_SIZE, drop_remainder=True)
    )

def prepare_valset(dataset):
    return (
        dataset
        .cache()
        .repeat()
        .batch(BATCH_SIZE, drop_remainder=True)
    )

train_dataset = prepare_trainset(train_dataset)
val_dataset = prepare_valset(val_dataset)

In [ ]:
input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]

In [ ]:
print(input_shape)
print(output_shape)

In [ ]:
if use_tpu:
  with strategy.scope():
    model = build_CONV_LSTM_model(input_shape, output_shape)
else:
  model = build_CONV_LSTM_model(input_shape, output_shape)

In [ ]:
# Train the Model
history = model.fit(
    train_dataset,
    steps_per_epoch=len(X_train) // BATCH_SIZE,
    epochs=300,
    validation_data=val_dataset,
    validation_steps=len(X_test) // BATCH_SIZE,
    callbacks=[
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=15, restore_best_weights=True, min_delta=1e-4),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)
    ]
).history

**Plot Loss & Validation Loss**

In [ ]:
best_epoch = np.argmin(history['val_loss'])
plt.figure(figsize=(17, 4))
plt.plot(history['loss'], label='Training loss', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation loss', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Squared Error')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(18, 3))
plt.plot(history['lr'], label='Learning Rate', alpha=.8, color='#ff7f0e')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.legend()
plt.grid(alpha=.3)
plt.show()

**Test**

In [ ]:
reg_predictions = np.array([])
X_temp = X_test
for reg in range(0, TEL, ATL):
    pred_temp = model.predict(X_temp, verbose=0)
    if (len(reg_predictions) == 0):
        reg_predictions = pred_temp
    else:
        reg_predictions = np.concatenate((reg_predictions, pred_temp), axis=1)
    X_temp = np.concatenate((X_temp[:, ATL:, :], pred_temp), axis=1)

In [ ]:
# Print the shape of the predictions
print(f"Predictions shape: {reg_predictions.shape}")
print()

print("Prediction at 18:")
mean_squared_error = tfk.metrics.mean_squared_error(y_test.flatten(), reg_predictions.flatten()).numpy()
print(f"Mean Squared Error: {mean_squared_error}")
mean_absolute_error = tfk.metrics.mean_absolute_error(y_test.flatten(), reg_predictions.flatten()).numpy()
print(f"Mean Absolute Error: {mean_absolute_error}")
print()

print("Prediction at 9:")
y_test_9 = y_test[:, :9]
reg_predictions_9 = reg_predictions[:, :9]
mean_squared_error = tfk.metrics.mean_squared_error(y_test_9.flatten(), reg_predictions_9.flatten()).numpy()
print(f"Mean Squared Error: {mean_squared_error}")
mean_absolute_error = tfk.metrics.mean_absolute_error(y_test_9.flatten(), reg_predictions_9.flatten()).numpy()
print(f"Mean Absolute Error: {mean_absolute_error}")
print()

**Save the Model**

In [ ]:
NAME_MODEL = "BidirectionalGRU"
if use_tpu:
  # save model locally from tpu using Tensorflow's "SavedModel" format
  save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
  model.save(NAME_MODEL, options=save_locally)
else:
  model.save(NAME_MODEL)